# Model Validation Code Along

For this code along, we will move through the various examples of model validation, demonstrating different ways of performing it.  We will move from a basic train-test split towards the most rigorous method using a pipeline.  

The overall goal of validation is to assess how your model will perform on unseen data.  The key to doing so will be to compare how well the model performs on data it has been fit on and data that it has not been fit on.   We select some metric appropriate to the problem (R^2, RMSE, etc), and look at the performance.  We then can say:  
  
    1: It is a low bias, high variance model if it performs well on the training, but poorly on the test. 
    2: It is high bias low variance if it performs poorly on the train and test.    
    3. It is low bias, low variance if it performs well on both training and test.  



Let's load in a data set with a continuous variable.

In [ ]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [ ]:
data = load_boston()
X = pd.DataFrame(data['data'])
X.columns = data['feature_names'] 
y = data['target']

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
df = X.copy()
df['target'] = y

In [ ]:
df.corr()['target']

# Basic Train-Test Split

The simplest form of model validation is performing a single train test split and comparing performance on training and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Create a split

In this version, the model building process would include testing hypotheses about features to include and transformations, and judging which model had the best R^2 on the test set. Moreover, we can judge overfitness by looking at the difference between the train and test score.

In [ ]:
# First we fit a linear model to the data set using. Only RM/LSTAT

In [ ]:
# Look at the score on the train set

In [ ]:
# Then we score the model on the test set

In [ ]:
# Compare training and test scores

Notice, we do not fit our model on the test set.  This concept will be repeated in many forms.  Do not fit on your test set.

We would continue trying out different hypotheses, and selecting the model that works best on the test set.  The issue becomes that in this process, every time we score our data on our test set, we are actually introducing bias into the model.  We are creating a hypothesis based on knowledge of how it will perform on the test set.   Adjusting it each time until we see that the test score is high. 

# Best Practice: split off test set and don't touch until the end

Best practice is to split off a portion of the data and not touch it until the very end.  None of your model-building will involve this hold out set.  It's only purpose will be as a final check to make sure that the model performs on unsean data as you would expect.

Let's split the data once more:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2)

Now, we will "put" the test set away, and not touch it until the very end.

# Secondary Train Test Split

Our test set allowed us to see the drop-off of performance between data our model has been trained on, and data it has not been trained on.Without the test set.  Now that we have set it aside, we need another way to judge whether our model is overfit.  

The simplest way to do that is by performing a secondary split, and building our model in the same way above, as if the training set were the only data in existence.

In [ ]:
X_tt, X_val, y_tt, y_val = train_test_split(X_train, y_train, random_state = 42, test_size = .2)

In [ ]:
lr = LinearRegression()
lr.fit(X_tt[['RM', 'LSTAT']], y_tt)
lr.score(X_tt[['RM', 'LSTAT']], y_tt)

In [ ]:
lr.score(X_val[['RM', 'LSTAT']], y_val)

# Transformations

If we were to perform some transformations to our model, we would have to be careful.  
Take for example a standard scaler.    

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()


A standard scaler learns the mean and standard deviation of our dataset in order to convert each data point into a z-score.

If we fit the data on the RM column, and include all of our data, the mean will be different than if we fit on X_train.


In [ ]:
ss.fit(X[['RM']])
ss.mean_

In [ ]:
ss.fit(X_train[['RM']])
ss.mean_

Therefore, if we fit our standard scaler on the whole data, then used it to transform our train set, we would be introducing information from the test set into the training set.  This is a form of data leakage.

To prevent this, we only ever fit our transformations (scalers, one-hot-encoders, upsamplers/downsamplers), on the training set.  

This applies even within the secondary split. 

So, with our secondary split example, we would proceed as follows:

In [ ]:
# Fit both our scalar and our model on the train set

In [ ]:
# Transform the validation set with the fit scaler
# Do not refit the scaler


Consider the following case: our best model includes every feature scaled.  

In [ ]:
# Fit the scaler on your training



In [ ]:
# Fit and score on training set

In [ ]:
# transform and score on the validation set

Since this is what we hypothesize to be our best model, we want to test it out on the hold out set.

To do so, we have to transform the test data exactly as we transformed our training data. In this example, our model was trained on scaled training features, so we have to scale the test features, or else our model will perform horribly.  

However, we want to train on as much information in the training set as we can.  So, once we have selected our final model, we then refit everything on the entire training set.

In [ ]:
# Fit on entire training

In [ ]:
# Transform holdout

# K-Folds



K-folds validation is an even more rigorous manner of performing validation.  It allows us to shuffle our training data into a specified number of splits.  Each time, a new portion of the data is held out as our validation set. 

We perform our k-folds within the training set.  And each time we fit our model, we have to be careful to only fit on the training portion.  This can get tricky.

In [ ]:
from sklearn.model_selection import train_test_split, KFold
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# instantiate a kfold object and specify number of splits
# designate 4 folds for each split

# loop through each fold
    
    # instantiate a Linear Regression object for each fold
    
    # instantiate a scaler for each fold
    
    # using the indices, create the split associated with each loop
    
    # fit transform the scaler on tt
    
    # fit model on tt
    
    # score both training and validation
    

In [ ]:
# Refit on entire training set

In [ ]:
# Score on holdout

# Pipelines

The process of fitting a KFold instance can be tricky. It allows for a lot of control, but can also lead to errors.  You can also forget to apply a transformation to the test set.  Pipelines allow you to package the transformations in one place.  

They also can be applied within cross validation and grid search methods (you will learn about grid search  soon).   

The same process as above of fitting the scaler on multiple subsets of the training set, and scoring on multiple val sets, can be performed in a couple of lines.


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

pipe = make_pipeline(StandardScaler(), LinearRegression())

cross_val_score(pipe, X_train, y_train, cv=4)


In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

In [ ]:
y_hat = pipe.predict(X_test)